In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from google.colab import drive
from keras.layers import LSTM
import matplotlib.pyplot as plt
import math

drive.mount('/content/gdrive')
from sklearn.metrics import mean_squared_error

look_back = 5
columns = 2
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back][0])
    return np.array(dataX), np.array(dataY)


Mounted at /content/gdrive


In [49]:
# file loader
data1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/stock_dataset/rnn/Cluster0_train.CSV',index_col="Date",encoding='cp949')
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/stock_dataset/rnn/Cluster0_test.CSV',index_col="Date",encoding='cp949')

volatility_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/stock_dataset/rnn/volatility_train.csv',index_col=0,encoding='cp949')

volatility_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/stock_dataset/rnn/volatility_test.csv',index_col=0,encoding='cp949')
volatility_train=volatility_train.T
volatility_test=volatility_test.T


In [52]:
col=list(data1.columns)

In [53]:
col

['두산중공업',
 '쌍용자동차',
 'KG동부제철',
 '에이프로젠제약',
 '현대일렉트릭',
 '한진칼',
 'SK디스커버리',
 '이월드',
 '신성이엔지',
 '에이프로젠 KIC',
 '신성통상',
 'STX중공업',
 '윌비스',
 '코오롱머티리얼',
 'SK케미칼',
 '오리엔트바이오',
 '세하',
 '현대에너지솔루션',
 '남선알미늄',
 '금호에이치티',
 '일진홀딩스',
 '일양약품',
 '효성중공업',
 'KR모터스',
 '진원생명과학',
 '센트럴인사이트',
 '엔케이물산',
 '알루코',
 '깨끗한나라',
 '동방',
 '부광약품',
 'HSD엔진',
 '한성기업',
 '삼성제약',
 '한국프랜지공업',
 'YG PLUS',
 '파미셀',
 '한창제지',
 '인디에프',
 '대웅',
 '동화약품',
 '신풍제약',
 '한올바이오파마',
 '코스모화학',
 '조비',
 '우리들휴브레인',
 '고려산업',
 '범양건영',
 '국보',
 '퍼스텍',
 '서원',
 '영화금속',
 'TCC스틸',
 '웰바이오텍',
 '백광산업',
 '금양',
 '세진중공업',
 '유니온머티리얼',
 '디피씨',
 '이엔플러스',
 '국동',
 '모나미',
 '모나리자',
 '국제약품',
 '써니전자',
 '남성',
 'CJ씨푸드',
 '페이퍼코리아',
 '신풍제지',
 '한창',
 '명문제약',
 '덕양산업',
 '한국특수형강',
 '화인베스틸',
 '형지엘리트',
 '인터지스',
 '대양금속',
 '세우글로벌',
 '한솔로지스틱스',
 '일진다이아',
 '일진디스플',
 '하이트론씨스템즈',
 '갤럭시아에스엠',
 '코리아써키트',
 '대호에이엘',
 '세종공업',
 '체시스',
 '서연',
 '화천기계',
 '잇츠한불',
 '주연테크',
 '경농',
 '삼화전자공업',
 '티비에이치글로벌',
 '부국철강',
 '키다리스튜디오',
 '유유제약',
 '덕성',
 '유나이티드',
 '한국화장품',
 '제일파마홀딩스',
 '유니온',
 '인지컨트롤스',
 '우리들

In [45]:
pandf_train['garch'][19]

2.9892470689999997

In [58]:
col=list(data1.columns)
mape_val=[]
for company in col:
  pandf_train = pd.DataFrame(data1, columns=[ company , "뉴스빈도"])
  pandf_train['garch']=volatility_train[company]
  pandf_train=pandf_train.iloc[20:]
  pandf_test = pd.DataFrame(data2, columns=[ company , "뉴스빈도"])
  pandf_test['garch']=volatility_test[company]
  print(pandf_train)
  # convert nparray # split train, test
  train = pandf_train.values
  train.astype('float32')
  test = pandf_test.values
  test.astype('float32')
  print(len(train), len(test))

  # create dataset for learning
  trainX, trainY = create_dataset(train, look_back)
  testX, testY = create_dataset(test, look_back)
  print(testY)
  # reshape input to be [samples, time steps, features]
  trainX = np.reshape(trainX, (trainX.shape[0], look_back, 3))
  testX = np.reshape(testX, (testX.shape[0], look_back,3))



  # simple lstm network learning
  model = Sequential()
  model.add(LSTM(5, input_shape=(look_back, 3)))   

  model.add(Dense(1,activation='tanh'))
  model.compile(loss='mean_squared_error', optimizer='rmsprop')
  model.fit(trainX, trainY, epochs=50, batch_size=2, verbose=2)

  #print(testPredict)
  #print(testY)
  testPredict = model.predict(testX)
  testScore = math.sqrt(mean_squared_error(testY, testPredict))
  sum=0
  for i in range(len(testY)):
    sum+=abs((testY[i]-testPredict[i])/testY[i])

  mape=100*sum/int(len(testY))
  print('MAPE :',float(mape))
  mape_val.append(mape)

m_val=[]
for i in mape_val:
  if i==float('inf'):
    m_val.append(0)
  else:
    m_val.append(float(i))

evaluate=pd.DataFrame({'mape':m_val},index=col)

print(evaluate)
print(evaluate.describe())

               두산중공업      뉴스빈도      garch
Date                                     
2020-01-03  0.017452  0.000147   1.731403
2020-01-06  0.034965  0.000905   2.625214
2020-01-07  0.021429  0.001222   3.115023
2020-01-08  0.050633  0.001051   3.048653
2020-01-09  0.022599  0.004694   3.626094
...              ...       ...        ...
2020-09-23  0.067616  0.804401   6.439094
2020-09-24  0.100000  0.794621  12.684014
2020-09-25  0.052830  0.584352   3.429350
2020-09-28  0.089494  0.775061   6.705399
2020-09-29  0.051095  0.564792  10.780798

[186 rows x 3 columns]
186 19
[0.0481539  0.04182208 0.08163265 0.05957447 0.12653061 0.11985019
 0.05617978 0.04444444 0.03649635 0.05734767 0.0703125  0.04850746
 0.05970149 0.05964912]
Epoch 1/50
91/91 - 0s - loss: 0.0587
Epoch 2/50
91/91 - 0s - loss: 0.0093
Epoch 3/50
91/91 - 0s - loss: 0.0067
Epoch 4/50
91/91 - 0s - loss: 0.0057
Epoch 5/50
91/91 - 0s - loss: 0.0050
Epoch 6/50
91/91 - 0s - loss: 0.0048
Epoch 7/50
91/91 - 0s - loss: 0.0044
Epoch 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide


Epoch 1/50
91/91 - 0s - loss: 0.0579
Epoch 2/50
91/91 - 0s - loss: 0.0082
Epoch 3/50
91/91 - 0s - loss: 0.0049
Epoch 4/50
91/91 - 0s - loss: 0.0039
Epoch 5/50
91/91 - 0s - loss: 0.0036
Epoch 6/50
91/91 - 0s - loss: 0.0034
Epoch 7/50
91/91 - 0s - loss: 0.0032
Epoch 8/50
91/91 - 0s - loss: 0.0031
Epoch 9/50
91/91 - 0s - loss: 0.0031
Epoch 10/50
91/91 - 0s - loss: 0.0029
Epoch 11/50
91/91 - 0s - loss: 0.0028
Epoch 12/50
91/91 - 0s - loss: 0.0029
Epoch 13/50
91/91 - 0s - loss: 0.0027
Epoch 14/50
91/91 - 0s - loss: 0.0028
Epoch 15/50
91/91 - 0s - loss: 0.0027
Epoch 16/50
91/91 - 0s - loss: 0.0026
Epoch 17/50
91/91 - 0s - loss: 0.0027
Epoch 18/50
91/91 - 0s - loss: 0.0026
Epoch 19/50
91/91 - 0s - loss: 0.0027
Epoch 20/50
91/91 - 0s - loss: 0.0027
Epoch 21/50
91/91 - 0s - loss: 0.0026
Epoch 22/50
91/91 - 0s - loss: 0.0025
Epoch 23/50
91/91 - 0s - loss: 0.0025
Epoch 24/50
91/91 - 0s - loss: 0.0025
Epoch 25/50
91/91 - 0s - loss: 0.0025
Epoch 26/50
91/91 - 0s - loss: 0.0024
Epoch 27/50
91/91 - 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
91/91 - 0s - loss: 0.0019
Epoch 33/50
91/91 - 0s - loss: 0.0019
Epoch 34/50
91/91 - 0s - loss: 0.0019
Epoch 35/50
91/91 - 0s - loss: 0.0019
Epoch 36/50
91/91 - 0s - loss: 0.0020
Epoch 37/50
91/91 - 0s - loss: 0.0020
Epoch 38/50
91/91 - 0s - loss: 0.0019
Epoch 39/50
91/91 - 0s - loss: 0.0019
Epoch 40/50
91/91 - 0s - loss: 0.0019
Epoch 41/50
91/91 - 0s - loss: 0.0019
Epoch 42/50
91/91 - 0s - loss: 0.0020
Epoch 43/50
91/91 - 0s - loss: 0.0020
Epoch 44/50
91/91 - 0s - loss: 0.0019
Epoch 45/50
91/91 - 0s - loss: 0.0019
Epoch 46/50
91/91 - 0s - loss: 0.0019
Epoch 47/50
91/91 - 0s - loss: 0.0019
Epoch 48/50
91/91 - 0s - loss: 0.0019
Epoch 49/50
91/91 - 0s - loss: 0.0019
Epoch 50/50
91/91 - 0s - loss: 0.0019
MAPE : 61.82099151611328
               일진다이아      뉴스빈도       garch
Date                                      
2020-01-03  0.025090  0.000147    2.214863
2020-01-06  0.029197  0.000905    2.247985
2020-01-07  0.019892  0.001222    1.985819
2020-01-08  0.

KeyError: ignored

In [59]:
m_val=[]
for i in mape_val:
  if i==float('inf'):
    m_val.append(0)
  else:
    m_val.append(float(i))

evaluate=pd.DataFrame({'mape':m_val},index=col[:-1])

print(evaluate)
print(evaluate.describe())

               mape
두산중공업     35.902012
쌍용자동차     64.803459
KG동부제철   117.777893
에이프로젠제약   28.966028
현대일렉트릭    24.865946
...             ...
삼성출판사     32.861412
진양폴리우레탄   50.616924
MH에탄올     87.679764
센트랄모텍     97.511635
두산솔루스     54.340855

[120 rows x 1 columns]
             mape
count  120.000000
mean    61.321741
std     29.145601
min      0.000000
25%     43.041636
50%     56.395733
75%     72.574974
max    164.016388


In [48]:
#evaluate=evaluate.drop(['뉴스빈도']) #기존 mape = 54
evaluate.describe()

NameError: ignored